In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>")) 

import os
import random
import numpy as np
import pandas as pd

In [2]:
def load_imdb_sentiment_analysis_dataset(data_path, seed=123):
    imdb_data_path = os.path.join(data_path, 'aclImdb')
    print(imdb_data_path)

    # Load the training data
    train_texts = []
    train_labels = []
    for category in ['pos', 'neg']:
        train_path = os.path.join(imdb_data_path, 'train', category)
        for fname in sorted(os.listdir(train_path)):
            if fname.endswith('.txt'):
                with open(os.path.join(train_path, fname)) as f:
                    train_texts.append(f.read())
                train_labels.append(0 if category == 'neg' else 1)

    # Load the validation data.
    test_texts = []
    test_labels = []
    for category in ['pos', 'neg']:
        test_path = os.path.join(imdb_data_path, 'test', category)
        for fname in sorted(os.listdir(test_path)):
            if fname.endswith('.txt'):
                with open(os.path.join(test_path, fname)) as f:
                    test_texts.append(f.read())
                test_labels.append(0 if category == 'neg' else 1)

    # Shuffle the training data and labels.
    random.seed(seed)
    random.shuffle(train_texts)
    random.seed(seed)
    random.shuffle(train_labels)

    return ((train_texts, np.array(train_labels)),
            (test_texts, np.array(test_labels)))



## using the word embeedings

In [7]:
from tensorflow.python.keras.preprocessing import sequence
from tensorflow.python.keras.preprocessing import text

def sequence_vectorize(train_texts, val_texts):
    # Vectorization parameters
    # Limit on the number of features. We use the top 20K features.
    TOP_K = 20000
    # Limit on the length of text sequences. Sequences longer than this
    # will be truncated.
    MAX_SEQUENCE_LENGTH = 500
    # Create vocabulary with training texts.
    tokenizer = text.Tokenizer(num_words=TOP_K)
    tokenizer.fit_on_texts(train_texts)

    # Vectorize training and validation texts.
    x_train = tokenizer.texts_to_sequences(train_texts)
    x_val = tokenizer.texts_to_sequences(val_texts)

    # Get max sequence length.
    max_length = len(max(x_train, key=len))
    if max_length > MAX_SEQUENCE_LENGTH:
        max_length = MAX_SEQUENCE_LENGTH

    # Fix sequence length to max value. Sequences shorter than the length are
    # padded in the beginning and sequences longer are truncated
    # at the beginning.
    x_train = sequence.pad_sequences(x_train, maxlen=max_length)
    x_val = sequence.pad_sequences(x_val, maxlen=max_length)
    return x_train, x_val, tokenizer.word_index,tokenizer


    

/home/rohit/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/rohit/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/rohit/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/rohit/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.

In [3]:
#load the data
# get the sequences
#define useful global variables
TOP_K = 20000
MAX_SEQUENCE_LENGTH  = 500
EMBEDDING_DIM = 100

data_path = '/home/rohit/Documents/Study/Projects/HACKATHON INNOVATE FOR IIT/aclImdb_v1'
(train_texts, train_labels), (test_texts, test_labels) = load_imdb_sentiment_analysis_dataset(data_path)

#sequences
x_train, x_test, tokenizer_word_index,tokenizer = sequence_vectorize(train_texts, test_texts)


num_words = min(TOP_K, len(tokenizer_word_index) + 1)

NameError: name 'load_imdb_sentiment_analysis_dataset' is not defined

In [ ]:
import pickle

# saving
with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)


In [91]:
train_texts[0]

"POSSIBLE SPOILERS<br /><br />The Spy Who Shagged Me is a muchly overrated and over-hyped sequel. International Man of Mystery came straight out of the blue. It was a lone star that few people had heard of. But it was stunningly original, had sophisticated humour and ample humour, always kept in good taste, and had a brilliant cast. The Spy Who Shagged Me was a lot more commercially advertised and hyped about.<br /><br />OK I'll admit, the first time I saw this film I thought it was very funny, but it's only after watching it two or three times that you see all the flaws. The acting was OK, but Heather Graham cannot act. Her performance didn't seem very convincing and she wasn't near as good as Liz Hurley was in the first one. Those characters who bloomed in the first one, (Scott Evil, Number 2 etc.) are thrown into the background hear and don't get many stand-alone scenes. The film is simply overrun with cameos.<br /><br />In particular, I hated the way they totally disregarded some o

In [5]:
#get the glove embeedings
glove_dir = '/home/rohit/Documents/Study/Projects/HACKATHON INNOVATE FOR IIT/'
embeddings_index = {}
with open(os.path.join(glove_dir, 'glove.6B.100d.txt')) as f:
    for line in f:
        word, coefs = line.split(maxsplit=1)
        coefs = np.fromstring(coefs, 'f', sep=' ')
        embeddings_index[word] = coefs

print('Found %s word vectors.' % len(embeddings_index))

Found 400000 word vectors.


In [6]:
embedding_matrix = np.zeros((num_words, EMBEDDING_DIM))
for word, i in tokenizer_word_index.items():
    if i >= TOP_K:
        continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector


In [9]:
#keras layers import
import keras
import tensorflow as tf
from keras.models import load_model
from keras.layers import Embedding
from keras.initializers import Constant
from keras import models, initializers, regularizers
from keras.layers import Dense, Dropout, SeparableConv1D, MaxPooling1D
from keras.layers import GlobalAveragePooling1D

Using TensorFlow backend.


    """Creates an instance of a separable CNN model.

    # Arguments
        blocks: int, number of pairs of sepCNN and pooling blocks in the model.
        
        filters: int, output dimension of the layers.
        
        kernel_size: int, length of the convolution window.
        
        embedding_dim: int, dimension of the embedding vectors.
        
        dropout_rate: float, percentage of input to drop at Dropout layers.
        
        pool_size: int, factor by which to downscale input at MaxPooling layer.
        
        input_shape: tuple, shape of input to the model.
        
        num_classes: int, number of output classes.
        
        num_features: int, number of words (embedding input dimension).
        
        use_pretrained_embedding: bool, true if pre-trained embedding is on.
        
        is_embedding_trainable: bool, true if embedding layer is trainable.
        
        embedding_matrix: dict, dictionary with embedding coefficients.

    # Returns
        A sepCNN model instance.
    """

In [19]:
def sepcnn_model(blocks,
                 filters = 64,
                 kernel_size = 3,
                 embedding_dim = EMBEDDING_DIM, 
                 dropout_rate = 0.2,
                 pool_size =2, 
                 MAX_SEQUENCE_LENGTH = MAX_SEQUENCE_LENGTH,
                 num_classes = 2,
                 num_features = num_words,
                 use_pretrained_embedding=True,
                 is_embedding_trainable=True,
                 embedding_matrix= embedding_matrix):
    
    op_units, op_activation = 1, 'sigmoid'
    model = models.Sequential()

    # Add embedding layer. If pre-trained embedding is used add weights to the
    # embeddings layer and set trainable to input is_embedding_trainable flag.
    if use_pretrained_embedding:
        model.add(Embedding(input_dim=num_features,
                            output_dim=embedding_dim,
                            input_length=MAX_SEQUENCE_LENGTH,
                            weights=[embedding_matrix],
                            trainable=is_embedding_trainable))
    else:
        model.add(Embedding(input_dim=num_features,
                            output_dim=embedding_dim,
                            input_length=input_shape[0]))

    for _ in range(blocks-1):
        model.add(Dropout(rate=dropout_rate))
        
        model.add(SeparableConv1D(filters=filters,
                                  kernel_size=kernel_size,
                                  activation='relu',
                                  bias_initializer='random_uniform',
                                  depthwise_initializer='random_uniform',
                                  padding='same'))
        
        model.add(SeparableConv1D(filters=filters,
                                  kernel_size=kernel_size,
                                  activation='relu',
                                  bias_initializer='random_uniform',
                                  depthwise_initializer='random_uniform',
                                  padding='same'))
        model.add(MaxPooling1D(pool_size=pool_size))

    model.add(SeparableConv1D(filters=filters * 2,
                              kernel_size=kernel_size,
                              activation='relu',
                              bias_initializer='random_uniform',
                              depthwise_initializer='random_uniform',
                              padding='same'))
    model.add(SeparableConv1D(filters=filters * 2,
                              kernel_size=kernel_size,
                              activation='relu',
                              bias_initializer='random_uniform',
                              depthwise_initializer='random_uniform',
                              padding='same'))
    model.add(GlobalAveragePooling1D())
    model.add(Dropout(rate=dropout_rate))
    model.add(Dense(op_units, activation=op_activation))
    return model


In [20]:
temp = sepcnn_model(blocks= 5)
temp.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_6 (Embedding)      (None, 500, 100)          2000000   
_________________________________________________________________
dropout_14 (Dropout)         (None, 500, 100)          0         
_________________________________________________________________
separable_conv1d_25 (Separab (None, 500, 64)           6764      
_________________________________________________________________
separable_conv1d_26 (Separab (None, 500, 64)           4352      
_________________________________________________________________
max_pooling1d_10 (MaxPooling (None, 250, 64)           0         
_________________________________________________________________
dropout_15 (Dropout)         (None, 250, 64)           0         
_________________________________________________________________
separable_conv1d_27 (Separab (None, 250, 64)          

In [21]:
learning_rate=1e-3,
epochs=1000,
batch_size= 128,
blocks=2,
filters=64,
dropout_rate= 0.2,
embedding_dim=EMBEDDING_DIM,
kernel_size=3,
pool_size=3

num_classes = 2
num_features = num_words

# Create model instance.
model = sepcnn_model(blocks = 2)
model.summary()


Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_7 (Embedding)      (None, 500, 100)          2000000   
_________________________________________________________________
dropout_19 (Dropout)         (None, 500, 100)          0         
_________________________________________________________________
separable_conv1d_35 (Separab (None, 500, 64)           6764      
_________________________________________________________________
separable_conv1d_36 (Separab (None, 500, 64)           4352      
_________________________________________________________________
max_pooling1d_14 (MaxPooling (None, 250, 64)           0         
_________________________________________________________________
separable_conv1d_37 (Separab (None, 250, 128)          8512      
_________________________________________________________________
separable_conv1d_38 (Separab (None, 250, 128)         

In [22]:
loss = 'binary_crossentropy'
optimizer = keras.optimizers.Adam(lr=learning_rate)
model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])

In [28]:
# Train and validate model.
history = model.fit(x_train, train_labels, epochs=5, verbose=1,batch_size= 128)

Epoch 1/5
25000/25000 [==============================] - 9s 373us/step - loss: 0.0038 - acc: 0.9986
Epoch 2/5
25000/25000 [==============================] - 9s 370us/step - loss: 0.0045 - acc: 0.9987
Epoch 3/5
25000/25000 [==============================] - 9s 369us/step - loss: 0.0042 - acc: 0.9988
Epoch 4/5
25000/25000 [==============================] - 9s 372us/step - loss: 0.0078 - acc: 0.9977
Epoch 5/5
25000/25000 [==============================] - 9s 374us/step - loss: 0.0031 - acc: 0.9990


In [32]:
#history = history.history
history.keys()
print(history['loss'], '\n',history['acc'])


[0.0037903949594311416, 0.004528580584379379, 0.004210526033178903, 0.007767128161918372, 0.003050965077120345] 
 [0.99856, 0.99872, 0.99884, 0.99772, 0.99904]


In [33]:
 # Save model.
model.save('IMDBmodel.h5')

In [34]:
model.evaluate(x_test, test_labels)

25000/25000 [==============================] - 3s 111us/step


[1.10974202296271, 0.86904]

In [48]:
for i in range(100):
    t1 = x_test[i]
    t1 = t1.reshape(1,t1.shape[0])
    print(t1.shape)
    if model.predict(t1)>0.6:
        print('positive')
    elif model.predict(t1)<= 0.4:
        print('negative')
    else:
        print('model is not sure. in neutral state')



(1, 500)
positive
(1, 500)
positive
(1, 500)
positive
(1, 500)
positive
(1, 500)
positive
(1, 500)
positive
(1, 500)
positive
(1, 500)
positive
(1, 500)
positive
(1, 500)
positive
(1, 500)
positive
(1, 500)
positive
(1, 500)
positive
(1, 500)
positive
(1, 500)
negative
(1, 500)
positive
(1, 500)
positive
(1, 500)
negative
(1, 500)
positive
(1, 500)
positive
(1, 500)
positive
(1, 500)
positive
(1, 500)
positive
(1, 500)
positive
(1, 500)
positive
(1, 500)
positive
(1, 500)
negative
(1, 500)
negative
(1, 500)
positive
(1, 500)
positive
(1, 500)
positive
(1, 500)
positive
(1, 500)
positive
(1, 500)
positive
(1, 500)
positive
(1, 500)
positive
(1, 500)
positive
(1, 500)
positive
(1, 500)
negative
(1, 500)
positive
(1, 500)
positive
(1, 500)
positive
(1, 500)
negative
(1, 500)
positive
(1, 500)
positive
(1, 500)
positive
(1, 500)
positive
(1, 500)
positive
(1, 500)
positive
(1, 500)
positive
(1, 500)
positive
(1, 500)
positive
(1, 500)
positive
(1, 500)
positive
(1, 500)
positive
(1, 500)
p

In [6]:
def sequence_of_single_sentence(sentence, tokenizer, MAX_SEQUENCE_LENGTH = 500):
    sen = tokenizer.texts_to_sequences(sentence)
    return sequence.pad_sequences(sen, maxlen= MAX_SEQUENCE_LENGTH)



y = ['iI admit, the great majority of films released before say 1933 are just not for me. Of the dozen or so "major" silents I have viewed, one I loved (The Crowd), and two were very good (The Last Command and City Lights, that latter Chaplin circa 1931).<br /><br />So I was apprehensive about this one, and humor is often difficult to appreciate (uh, enjoy) decades later. I did like the lead actors, but thought little of the film.<br /><br />One intriguing sequence. Early on, the guys are supposed to get "de-loused" and for about three minutes, fully dressed, do some schtick. In the background, perhaps three dozen men pass by, all naked, white and black (WWI ?), and for most, their butts, part or full backside, are shown. Was this an early variation of beefcake courtesy of Howard Hughes?']

y= ['honey that movie was marvelous ']
y = [' today ']
y = ["William Boyd and Louis Wolheim are the ""Two Arabian Knights"" referred to in the title, humorously. The pair start out as U.S. POWs trying to escape from the Germans during World War I. Eventually, they find themselves on board a ship bound for Arabia. While tripping out to the Middle East, they rescue an Arab woman, Mary Astor, who turns out to be a Princess; and, of course, becomes a romantic interest for the Two Arabian Knights. No points for guessing who wins the veiled Ms. Astor!<br /><br />The film is very well photographed and directed; Lewis Milestone has wonderful sets, and stages scenes beautifully. Of the performances, Mr. Wolheim stands out - he creates a character so understandable you can almost hear him speak, trough the film is silent. The story isn't as strong as it could be - there are some events and sequences which had me wondering how and why the characters' locale changed. The last looks, exchanged between one of the stars and an extra, is an example of something I didnt understand. Perhaps these were comic bits which had a particular appeal for the time.<br /><br />The film is damaged in several places; but there is enough preserved, in even these scenes, to allow your mind to fill in the visual blanks. Boris Karloff appears as the Purser; watch for his big scene on ship, when Wolheim goes into a room with him for some money (what actually happens is a mystery). Early in the film, there is a long scene with a lot of naked men shown from the waist up (or, thereabouts); they are POWs being herded to the showers. Director Milestone uses parades of soldiers moving to great effect; this shower scene is different in that several of the men don't look as Caucasian as you might expect - maybe not as many Caucasian men would agree to appear nude? <br /><br />******* Two Arabian Knights (9/23/27) Lewis Milestone ~ William Boyd, Louis Wolheim, Mary Astor"]
y = ["worst"]
y = ["food was okay only. but i am sad.it's okay "]
y = ["i am not sad with food of our college"]
y = ['sad is that he died. it was boring']
y = ['Right now Im mostly just sprouting this so my cats can eat the grass. They love it. I rotate it around with Wheatgrass and Rye too']
y = ['I dont know if its the cactus or the tequila or just the unique combination of ingredients, but the flavour of this hot sauce makes it one of a kind! We picked up a bottle once on a trip we were on...']
y = ['Product arrived labeled as Jumbo Salted Peanuts...the peanuts were actually small sized unsalted. Not sure if this was an error or if the vendor intended to represent the product as "Jumbo".']
y = ['I love eating them and they are good for watching TV and looking at movies! It is not too sweet. I like to transfer them to a zip lock baggie so they stay fresh so I can take my time eating them.']
y = ['The candy is just red , No flavor . Just plan and chewy . I would never buy them again']
y = ['The flavors are good. However, I do not see any differce between this and Oaker Oats brand - they are both mushy']

y = ['Buyer Beware Please! This sweetener is not for everybody. Maltitol is an alcohol sugar and can be undigestible in the body. You will know a short time after consuming it if you are one of the unsusp...']
y = ['Food was very tasty. Though, they maintain less hygiene, but you can go for once and all for having tasty food while returning from tiring journey.']
y = ['mess secretary please improve']
k = sequence_of_single_sentence(y, tokenizer)
print(k.shape)



model.predict(k)

NameError: name 'tokenizer' is not defined

In [18]:
x = []
y = 'hello mess improve'
x.append(y)
k = sequence_of_single_sentence(x, tokenizer)
model.predict(k)

array([[0.03460965]], dtype=float32)

In [ ]:
from keras.models import load_model
model =getModelFunny()





Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.



In [3]:
import pickle

In [4]:
with open('tokenizer_on_new_train.pickle', 'rb') as handle:
    tokenizer = pickle.load(handle)

In [5]:
def getModelFunny():
    model = load_model('model_rnn_pretrained_trained.h5')
    return model
def getModelSentiment():
    return load_model('rnn_sentiment_86.h5')

In [ ]:
def sequence_of_single_sentence(sentence, tokenizer, MAX_SEQUENCE_LENGTH = 500):
    sen = tokenizer.texts_to_sequences(sentence)
    return sequence.pad_sequences(sen, maxlen= MAX_SEQUENCE_LENGTH)
Feedback = 'not good'
x = [str(Feedback)]

y = np.asarray(x)

k = sequence_of_single_sentence(y, tokenizer)

model_Sent = getModelSentiment()
print("model loaded")
out = model_Sent.predictSentiment(k)
print("out ",out)
sentiment_star = out[0][1]
print(sentiment_star)





Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.



In [2]:
import numpy as np
